In [15]:

import sys

#move path to parent directory
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

#load dataset here
from datasets.tency_mastering_multitrack_random import TencyMastering

import omegaconf

config = omegaconf.OmegaConf.load("../conf/dset/tencymastering_multitrack_4mapper_paired_server5.yaml")

import hydra

dset= hydra.utils.instantiate(config.train)

import torch

train_loader = torch.utils.data.DataLoader(dataset=dset, batch_size=1,
                                               num_workers=1, pin_memory=True, timeout=0, prefetch_factor=5)
train_loader = iter(train_loader)


from utils.AF_features_embedding_v4 import AF_fourier_embedding

device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
AF= AF_fourier_embedding(device=device, output_dim=128, sigma=0.2)

#from networks.FourierInverter import FourierInverter

#network= FourierInverter(AF.output_dim, AF.input_dim).to(device)

-1 num_examples


In [16]:
num_iterations= 200
for i in range(num_iterations):
    _, y= next(train_loader)
    y=y.to(device)

    with torch.no_grad():
        emb, features, flat_features = AF.encode(y)
        rms=features[0]
        est_features= AF.decode(emb)
        rms_est=est_features[0]

        error_rms= torch.abs(rms - rms_est).mean().item()
        print(error_rms)



9.5367431640625e-07
3.814697265625e-06
9.5367431640625e-07
9.5367431640625e-07
9.5367431640625e-07
1.9073486328125e-06
9.5367431640625e-07
0.0
9.5367431640625e-07
9.5367431640625e-07
0.0
9.5367431640625e-07
0.0
9.5367431640625e-07
2.86102294921875e-06
0.0
1.430511474609375e-06
0.0
1.9073486328125e-06
1.9073486328125e-06
9.5367431640625e-07
0.0
9.5367431640625e-07
9.5367431640625e-07
9.5367431640625e-07
9.5367431640625e-07
9.5367431640625e-07
9.5367431640625e-07
1.9073486328125e-06
1.9073486328125e-06
1.9073486328125e-06
0.0
0.0
0.0
9.5367431640625e-07
0.0
0.0
9.5367431640625e-07
9.5367431640625e-07
1.9073486328125e-06
0.0
1.9073486328125e-06
0.0
9.5367431640625e-07
4.76837158203125e-06
9.5367431640625e-07
0.0
0.0
9.5367431640625e-07
9.5367431640625e-07
0.0
0.0
9.5367431640625e-07
9.5367431640625e-07
1.9073486328125e-06
3.814697265625e-06
1.9073486328125e-06
0.0
3.814697265625e-06
9.5367431640625e-07
0.0
9.5367431640625e-07
9.5367431640625e-07
0.0
9.5367431640625e-07
1.9073486328125e-06

In [8]:

num_iterations= 10000
learning_rate = 0.001

optimizer = torch.optim.AdamW(network.parameters(), lr=learning_rate)
alpha = 0.7  # Weight for ground truth loss
beta = 0.3   # Weight for forward consistency loss

for i in range(num_iterations):
    _, y= next(train_loader)
    y=y.to(device)

    with torch.no_grad():
        emb, _, flat_features = AF.encode(y)

    #print("emb.shape", emb.shape)
    predicted_features = network(emb)

    loss_gt = torch.mean((predicted_features - flat_features) ** 2)

    # Forward consistency loss (not used in this example, but can be added)
    projected = predicted_features @ AF.projection
    cos_features = torch.cos(projected)
    sin_features = torch.sin(projected)
    rec_emb=torch.cat([cos_features, sin_features], dim=-1) * AF.scale_factor

    ##use cosine distance for forward consistency loss
    loss_forward =(1-torch.cosine_similarity(rec_emb, emb, dim=-1)).mean()

    loss= alpha * loss_gt + beta * loss_forward
    #loss=loss_gt


    loss_gt.backward()
    optimizer.step()
    optimizer.zero_grad()

    if (i + 1) % 10 == 0:
        print(f"Iteration {i+1}/{num_iterations}, Loss gt: {loss_gt.item()}, loss forward: {loss_forward.item()}, Total Loss: {loss.item()}")
        #print(f"Iteration {i+1}/{num_iterations}, Loss gt: {loss_gt.item()}")
        print("flat_features:", flat_features, "predicted_features:", predicted_features)









Iteration 10/10000, Loss gt: 0.6092067360877991, loss forward: 0.6425173878669739, Total Loss: 0.6191999316215515
flat_features: tensor([[ 1.4804e-01, -8.6568e-01, -4.8546e-01, -4.1751e-01,  6.4548e-01,
          6.4532e-01,  9.9110e-02, -1.9240e+00],
        [ 2.3571e-01,  2.3442e-01,  3.4127e-01,  3.5723e-01,  3.3231e-01,
          3.2408e-01, -7.0191e-01, -2.9538e-03],
        [-1.1570e+00, -3.7301e-01, -2.7615e-01, -1.5784e-01, -1.9966e-01,
         -2.3386e-01, -2.4436e-01,  1.5980e+00],
        [ 8.4791e-01,  8.5172e-01, -5.3491e-01, -5.8605e-01,  6.9360e-01,
          6.9959e-01, -6.5131e-01,  8.7581e-03],
        [ 3.3760e-01,  8.1064e-02, -2.3040e-01, -3.1620e-01,  7.0678e-01,
          7.0904e-01,  9.6621e-01, -5.8241e-01],
        [ 4.1550e-01,  4.1550e-01, -1.1491e+00, -1.1492e+00,  5.5434e-01,
          5.5431e-01, -7.1795e-01, -3.6503e-06],
        [-2.8255e-01,  1.5160e-01, -6.1326e-01, -2.6826e-01,  7.0357e-01,
          6.9258e-01,  1.4685e-02,  9.6077e-01],
        [ 

KeyboardInterrupt: 